In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 한국어 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'  # macOS
# plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
plt.rcParams['axes.unicode_minus'] = False

In [19]:
df= pd.read_csv("/Users/jang-wonjun/Desktop/Dev/Bulletin_board_analysis/outputs/final_data/final_data.csv")
major_topic_names_df = pd.read_csv("/Users/jang-wonjun/Desktop/Dev/Bulletin_board_analysis/outputs/major_topic_names/major_topic_names.csv")
topic_names_df = pd.read_csv("/Users/jang-wonjun/Desktop/Dev/Bulletin_board_analysis/outputs/topic_names/topic_names.csv")

In [20]:
major_topic_names_df.head(2)

,major_topic,major_topic_name
0,0,주택 관리 상담
1,1,관리규약 상담


In [21]:
topic_names_df.head(2)

,topic,topic_name
0,0_0,상가 운영 및 오피스텔 관리
1,0_1,아파트 관리인 및 위원회 문의


In [ ]:
df.head(2)

,상담일자,상담유형,상담요약,상담인 유형,상담내용,연도,major_topic_name,topic_name,major_topic,subtopic,topic,상담연도,상담월,상담일,상담분기,상담계절
0,2023.1.2-6,전화,누수피해,구분소유자,배수관 역류에 따라 바닥재(마루) 및 씽크대 등의 피해가 발생하였습니다. 역류원인을...,2023,누수 분쟁 해결,다세대빌라 누수분쟁,7,0,7_0,2023,1,2,1,겨울
1,2023.1.2-6,전화,통합정보마당,임차인,통합정보마당 사용 방법에 대한 이해,2023,건물 관리 및 운영,집합건물 상가 관리,8,3,8_3,2023,1,2,1,겨울


In [9]:
columns = ['상담일자', '상담유형', '상담요약', '상담인 유형', '상담내용', 'major_topic_name', 'topic_name', 'major_topic', 'subtopic', 'topic', '상담연도', '상담월', '상담일']

df = df[columns]

print("=== 데이터 기본 정보 ===")
print(df.info())
print("\n=== 결측치 확인 ===")
print(df.isna().sum())

print("\n=== 중복 행 수 ===")
print(df.duplicated().sum())

print("\n=== 주요 컬럼별 유니크 값 수 ===")
cols_to_check = ["연도", "major_topic", "major_topic_name", "subtopic", "topic", "topic_name"]
for col in cols_to_check:
    if col in df.columns:
        print(f"{col}: {df[col].nunique()}개")

print("\n=== major_topic vs major_topic_name 매핑 확인 ===")
if "major_topic" in df.columns and "major_topic_name" in df.columns:
    mapping_check = df.groupby("major_topic")["major_topic_name"].nunique()
    print(mapping_check)

print("\n=== major_topic별 데이터 수 ===")
if "major_topic_name" in df.columns:
    print(df["major_topic_name"].value_counts())

print("\n=== major_topic_name -> topic_name 계층 구조 확인 ===")
if "major_topic_name" in df.columns and "topic_name" in df.columns:
    for mt in df["major_topic_name"].unique():
        subtopics = df[df["major_topic_name"] == mt]["topic_name"].unique()
        print(f"\n{mt} ({len(subtopics)} 소주제):")
        for st in subtopics:
            print(f"  - {st}")

=== 데이터 기본 정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6940 entries, 0 to 6939
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   상담일자              6940 non-null   object
 1   상담유형              6938 non-null   object
 2   상담요약              6940 non-null   object
 3   상담인 유형            6937 non-null   object
 4   상담내용              6940 non-null   object
 5   major_topic_name  6940 non-null   object
 6   topic_name        6940 non-null   object
 7   major_topic       6940 non-null   int64 
 8   subtopic          6940 non-null   int64 
 9   topic             6940 non-null   object
 10  상담연도              6940 non-null   int64 
 11  상담월               6940 non-null   int64 
 12  상담일               6940 non-null   int64 
dtypes: int64(5), object(8)
memory usage: 705.0+ KB
None

=== 결측치 확인 ===
상담일자                0
상담유형                2
상담요약                0
상담인 유형              3
상담내용                0
ma

In [10]:
print(df['major_topic_name'].unique())
print(len(df['major_topic_name'].unique()))
print(df['major_topic_name'].value_counts())

['누수 분쟁 해결' '건물 관리 및 운영' '건물 관리 상담' '주택관리 운영' '분쟁조정 안내' '공용부분 관리'
 '주택 관리 상담' '관리규약 상담' '의결권 관리']
9
major_topic_name
건물 관리 상담      2316
건물 관리 및 운영    1931
공용부분 관리        668
분쟁조정 안내        492
주택관리 운영        453
관리규약 상담        451
누수 분쟁 해결       236
주택 관리 상담       217
의결권 관리         176
Name: count, dtype: int64


In [13]:
print("컬럼 갯수:", len(df.columns))
print("데이터 수:", len(df))
print("최종 컬럼들:", df.columns)

컬럼 갯수: 13
데이터 수: 6940
최종 컬럼들: Index(['상담일자', '상담유형', '상담요약', '상담인 유형', '상담내용', 'major_topic_name',
       'topic_name', 'major_topic', 'subtopic', 'topic', '상담연도', '상담월', '상담일'],
      dtype='object')


In [15]:
df.to_excel("/Users/jang-wonjun/Desktop/Dev/Bulletin_board_analysis/outputs/final_data/final_data.xlsx", index=False)